In [ ]:
import datetime
from dotenv import dotenv_values
import json
import pprint

from pymongo import MongoClient, GEO2D, GEOSPHERE

config = dotenv_values(".env")

In [ ]:
def get_database():
    client = MongoClient(config["CONNECTION_STRING"])

    return client['test']

def print_query_results(cursor):
    pprint.pprint(list(cursor))

In [ ]:
db = get_database()

users = db.users
print_query_results(users.find())    

## Find users with trust score at least 1000

In [ ]:
cursor = users.find({"trustScore": {"$gte": 1000}})
print_query_results(cursor)

## Add creation date and time to one account

In [ ]:
result = users.update_one(
    {"username": "akriese"}, 
    {"$set": {"creationTime": datetime.datetime(2023, 3, 20, 18, 1, 42)}}
)
result.matched_count, result.modified_count

## Create an index

In [ ]:
# uncomment if no index created yet
#users.create_index([("lastLocation.coordinates", GEO2D)])

## Find objects in close distance

In [ ]:
from bson.son import SON
radius = 0.01
print(f"With {radius=}:")
cursor = users.find({"lastLocation.coordinates": SON([("$near", [15.32, 50.49]), ("$maxDistance", radius)])})
print_query_results(cursor)

radius = 0.1
print(f"With {radius=}:")
cursor = users.find({"lastLocation.coordinates": SON([("$near", [15.32, 50.49]), ("$maxDistance", radius)])})
print_query_results(cursor)

## Find objects in bounding box

`"$box": [[long_lower_left, lat_lower_left], [long_upper_right, lat_upper_right]]`

In [ ]:
cursor = users.find({"lastLocation.coordinates": {"$within": {"$box": [[15.32, 50.49], [15.37, 50.51]]}}})
print_query_results(cursor)

## Inserting data from OSM (overpass) into locations

In [ ]:
data = json.load(open("sports.json", "r"))

In [ ]:
elements = data["elements"]

In [ ]:
# look how often tags occur in the data
from collections import Counter
tags = []
for el in elements:
    for t in el["tags"].keys():
        tags.append(t)
        
sorted(Counter(tags).items(), key=lambda x: x[1], reverse=True)

In [ ]:
import munch
def osm_to_mongo(loc):
    loc = munch.DefaultMunch.fromDict(loc) # easier access in nested dicts
    coords = [loc.lon, loc.lat] if loc.center is None else [loc.center.lon, loc.center.lat]
    d = {
        "activityType": loc.tags.sport,
        "location": {
            "type": loc.type,
            "coordinates": coords,
        },
        "osmId": loc.id,
        "tags": {k: v for k, v in loc.tags.items() if k != "sport"},
        "trustScore": 1000,
        "rating": 0,
    }
    if loc.tags.name is not None:
        d |= {"title": loc.tags.name}
    if loc.tags.description is not None:
        d |= {"description": loc.tags.description}
    return d

In [ ]:
item = osm_to_mongo(elements[0])

In [ ]:
locations = db.locations

In [ ]:
%%time
results = locations.insert_many([osm_to_mongo(loc) for loc in elements if loc["tags"]["sport"] != "table_tennis"])

In [ ]:
#result = locations.delete_many({})

In [ ]:
results.inserted_ids

## Create location index

In [ ]:
# uncomment if no index created yet
#locations.create_index([("location.coordinates", GEO2D)])

## Search locations close to one user

In [ ]:
u = users.find_one({"username": "akriese"})

In [ ]:
%%time
query = {"location.coordinates": {"$within": {"$center": [u["lastLocation"]["coordinates"], 0.1]}}, "activityType": "soccer"}
cursor = locations.find(query)
#len(list(cursor))

In [ ]:
%%time
for x in cursor:
    pprint.pprint(x)

## Use json schema

In [ ]:
user_schema = json.load(open("database/schemas/users.schema.json", "r"))

In [ ]:
print(len(list(users.find({"$jsonSchema": user_schema}))))